In [ ]:
from rapidfuzz import process as fuzz_process
from rapidfuzz import fuzz as fuzz_fuzz
from underthesea import word_tokenize
from LLM import Process_LLM
import unicodedata
import string
import csv
import re

def normalize_text(text):
    text = text.replace('đ', 'd').replace('Đ', 'D')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = text.lower().strip()
    return text
def extract_keywords(s):
    def is_number(s):
        return bool(re.fullmatch(r'[\d,. ]+', s))
    specialchars = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '..', '...', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    specialwords = ["giấy tờ", "thủ tục", "giấy", "gì", "cần", "nào", "sắp", "đang", "sẽ", "của", "bị", "hoặc", "với", "và", "thì", "muốn"]
    words = word_tokenize(s)
    words = [w.lower().strip() for w in words]
    words = list(set(words))
    words = [w for w in words if not is_number(w)]
    words = [w for w in words if w not in specialchars]
    words = [w for w in words if w not in specialwords]
    words_normalized = [normalize_text(w) for w in words if " " in w]
    return words + words_normalized

with open('url/cache', mode='r', newline='', encoding='utf-8') as f:
    thutucs = csv.DictReader(f)
    thutucs = sorted(thutucs, key=lambda e: len(e["Tên thủ tục"]))
    for i in range(len(thutucs)):
        thutucs[i]['keywords'] = extract_keywords(thutucs[i]['Tên thủ tục'])

In [ ]:
test_str = [
"sắp khởi nghiệp cần giấy tờ gì",
"tôi muốn mua đất thì cần gì",
"tôi sắp lập gia đình thì cần làm gì",
"vợ tôi sắp sinh con thủ tục nào",
"thủ tục xây nhà cấp 3, 4",
"phúc khảo bài thi thpt",
]

In [ ]:
for question in test_str:
    # ====================================================================================================
    prompt = f"""\
    Câu hỏi: "{question}".
    Tên của văn bản thủ tục hành chính quan trọng nhất liên quan đến câu hỏi trên là gì?
    Trả lời ngắn gọn, không dài dòng, không giải thích, không văn bản thừa.
    """
    llm_res = Process_LLM(prompt, vendor="openrouter").lower().strip(string.punctuation + string.whitespace)

    def find_the_best_thutucs(question):
        q_keywords = extract_keywords(question)
        thutuc_scores = []
        for e in thutucs:
            # count 1: keywords
            count = 0
            for k in q_keywords:
                if k in e["keywords"]:
                    count += 1
            # count 2: full name
            if normalize_text(question) in normalize_text(e["Tên thủ tục"]):
                count += 1
                if question.lower() in e["Tên thủ tục"].lower():
                    count += 1
            thutuc_scores.append(count)
        thutuc_scores_idx = sorted(range(len(thutuc_scores)), key=lambda i: thutuc_scores[i], reverse=True)
        scores = [(idx, thutuc_scores[idx]) for idx in thutuc_scores_idx]
        scores = [e for e in scores if e[1] != 0]
        # Return
        for i in range(min(len(scores), 3)):
            print(f"{thutucs[scores[i][0]]['Tên thủ tục']} (score={scores[i][1]})")

    print(f"question: {question}")
    print(f"llm_res: {llm_res}")
    print("-"*50)
    find_the_best_thutucs(question)
    print("-"*50)
    fuz_res = [(ee[2], ee[1]) for ee in fuzz_process.extract(llm_res, [e["Tên thủ tục"].lower().strip() for e in thutucs], scorer=fuzz_fuzz.token_set_ratio, limit=3)]
    for i in range(len(fuz_res)):
        print(f"{thutucs[fuz_res[i][0]]['Tên thủ tục']} (score={fuz_res[i][1]})")
    print("-"*50)
    find_the_best_thutucs(llm_res)
    print("="*100)
    # ====================================================================================================